In [1]:
import numpy as np

from keplergl import KeplerGl

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

In [2]:
#read_file = pd.read_csv(r'significant-earthquake-database.csv')
df=pd.read_table("earthquakes-2023.tsv",sep='\t')
#csv_table.to_csv('earthquakes-2023.csv',index=False)
#df = pd.read_csv('earthquakes-2023.csv')
display(df)

,Search Parameters,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,-2150.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,"JORDAN: BAB-A-DARAA,AL-KARAK",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,-2000.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SYRIA: UGARIT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,-2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURKMENISTAN: W,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
4,NaN,-1610.0,NaN,NaN,NaN,NaN,NaN,3.0,1351.0,GREECE: THERA ISLAND (SANTORINI),...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6347,NaN,2023.0,2.0,6.0,1.0,17.0,35.0,5873.0,NaN,TURKEY; SYRIA,...,NaN,NaN,118300.0,4.0,34200.0,4.0,174321.0,4.0,NaN,4.0
6348,NaN,2023.0,2.0,6.0,10.0,24.0,49.0,NaN,NaN,TURKEY; SYRIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6349,NaN,2023.0,2.0,9.0,6.0,28.0,0.0,NaN,NaN,INDONESIA: NEW GUINEA: IRIAN JAYA: JAYAPURA,...,NaN,NaN,NaN,NaN,NaN,1.0,15.0,1.0,28.0,1.0
6350,NaN,2023.0,2.0,20.0,17.0,4.0,29.0,NaN,NaN,TURKEY; SYRIA,...,NaN,NaN,200.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# preprocessing
# print all column names
# print(df.columns)
# create a new dataframe with 'Latitude', 'Longitude', 'Mag', 'Location Name'
df_seismic = df[['Latitude', 'Longitude', 'Mag', 'Location Name']]
# remove all columns with NaN values
df_seismic = df_seismic.dropna()
display(df_seismic)

,Latitude,Longitude,Mag,Location Name
1,31.100,35.500,7.3,"JORDAN: BAB-A-DARAA,AL-KARAK"
3,38.000,58.200,7.1,TURKMENISTAN: W
8,32.000,35.500,6.5,ISRAEL: ARIHA (JERICHO)
9,29.600,35.000,6.2,JORDAN: SW: TIMNA COPPER MINES
15,39.700,23.300,7.0,GREECE: MACEDONIA
...,...,...,...,...
6347,37.166,37.042,7.8,TURKEY; SYRIA
6348,38.024,37.203,7.5,TURKEY; SYRIA
6349,-2.635,140.557,5.1,INDONESIA: NEW GUINEA: IRIAN JAYA: JAYAPURA
6350,36.109,36.017,6.3,TURKEY; SYRIA


In [5]:
map = KeplerGl(height=600)
map.add_data(data=df_seismic, name='seismic')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [6]:
map.save_to_html(file_name='seismic.html')

Map saved to seismic.html!
